In [20]:
import pandas as pd
import os
from sklearn import tree

In [21]:
# x_train
training_data = pd.read_csv('../data/original_data/noExclusion_train_data.csv', header = None)
# y_train
training_labels = pd.read_csv('../data/original_data/noExclusion_train_label.csv', header = None)
# x_test
testing_data = pd.read_csv('../data/original_data/noExclusion_test_data.csv', header = None)
# y_test
testing_labels = pd.read_csv('../data/original_data/noExclusion_test_label.csv', header = None)


In [22]:
# make decision tree classifier 
training_data.shape
training_labels.shape

(572, 1)

In [23]:
clf_cart = tree.DecisionTreeClassifier(criterion="gini", random_state=1)
clf_cart = clf_cart.fit(training_data, training_labels)

In [27]:
# make predictions with CART model
# y_pred = clf_cart.predict(testing_data)
accuracy = clf_cart.score(testing_data, testing_labels)

print(accuracy)

0.7762237762237763
